# Machine vs Deep Learning

What is the difference? 

**Machine learning** is an algorithm that teaches a machine to think. More specifically, it is an algorith that can take in data, learn from that data and apply what they've learning to make informed decisions. 

**Deep learning** is machine learning. In other words deep learning is a subset of machine learning that mimics the way a human brain draws conclusions by inspiring from our neural network to create an artificial neural network. 

This section on deep learning uses *TensorFlow*, the most popular tool for deep learning and *Karis*, an API for specifying deep learning models. 

### Definition

__Tensor__: A matrix that can have a combination of any number of columns and rows.